In [1]:
class Compiler:
    
    def __init__(self,production):
        self.productions = production
        self.eleftrecusrion = []
        self.eleftfctoring = []
        self.finalgrammer = []

    def eliminate_left_recursion(self,productions):
        # Step 1: Identify left recursive productions
        left_recursive = []
        for production in productions:
            if production[1][0] == production[0]:
                left_recursive.append(production)

        # Step 2: Replace left recursive non-terminals with new non-terminals
        non_terminals = set([p[0] for p in productions])
        for production in left_recursive:
            new_nt = production[0] + "'"
            while new_nt in non_terminals:
                new_nt += "'"
            non_terminals.add(new_nt)

            # Step 3: Add new production for original non-terminal
            alternative = production[1][1:] + new_nt
            productions.append((production[0], alternative))

            # Step 4: Add new production for new non-terminal
            productions.append((new_nt, production[1] + new_nt))

        # Step 5: Remove original left recursive productions
        productions = [p for p in productions if p not in left_recursive]

        return productions
    
    
    def eliminate_left_factoring(self,productions):
        # Step 1: Identify left factored productions
        left_factored = []
        for production in productions:
            expansions = production[1].split("|")
            # Check if any two alternatives have a common prefix
            for i, a1 in enumerate(expansions[:-1]):
                for a2 in expansions[i+1:]:
                    common_prefix = ""
                    for j, c in enumerate(a1):
                        if c != a2[j]:
                            break
                        common_prefix += c
                    if len(common_prefix) > 0:
                        left_factored.append(production)
                        break

        # Step 2: Replace left factored productions with new productions
        for production in left_factored:
            common_prefix = ""
            alternatives = production[1].split("|")
            for i, c in enumerate(alternatives[0]):
                if all(a[i] == c for a in alternatives):
                    common_prefix += c
                else:
                    break
            
            # Add new productions for each alternative
            new_productions = []
            for alternative in alternatives:
                new_nt = f"{alternative}'"
                new_productions.append((production[0], f"{common_prefix}{new_nt}"))
                new_productions.append((new_nt, alternative[len(common_prefix):]))

            # Step 3: Add new productions
            productions += new_productions

        # Step 4: Remove original left factored productions
        productions = [p for p in productions if p not in left_factored]

        return productions



    # Calculate the FIRST sets for the grammar
    def first(self,productions):
        
        # Create a dictionary to store the first set for each non-terminal
        first_set = {non_terminal: set() for (non_terminal, _) in productions}

        # Add the terminal symbols to the first set
        terminals =  {symbol for (_, right_side) in productions for symbol in right_side if symbol.isalpha()} and {'+','*','(',')','-'}
        for terminal in terminals:
            first_set[terminal] = {terminal}

        # Iterate over the productions until the first set for each non-terminal stops changing
        while True:
            changes = False

            # Iterate over the productions
            for (non_terminal, right_side) in productions:
                # If the right side of the production is a terminal symbol, add it to the first set for the non-terminal
                if right_side[0].isalpha():
                    if right_side[0] not in first_set[non_terminal]:
                        first_set[non_terminal].add(right_side[0])
                        changes = True
                # If the right side of the production is a non-terminal symbol, add its first set to the first set for the non-terminal
                else:
                    new_elements = first_set[right_side[0]] - first_set[non_terminal]
                    if new_elements:
                        first_set[non_terminal].update(new_elements)
                        changes = True

            # If the first set for each non-terminal stopped changing, we are done
            if not changes:
                break

        return first_set


    # Calculate the FOLLOW sets for the grammar
    def follow(self,grammar, first_sets):
        follow_sets = {}
        for lhs, rhs in grammar:
            follow_sets[lhs] = set()
            for i, symbol in enumerate(rhs):
                if symbol.isupper():
                    follow_sets[symbol].update(follow_sets[lhs])
                    if "EPSILON" not in first_sets[symbol]:
                        break
                else:
                    follow_sets[symbol] = set(symbol)
                    break
        return follow_sets
     

    def create_parse_table(self,grammar, first, follow):
        parse_table = {}
        for lhs, rhs in grammar:
            parse_table[lhs] = {}
            for symbol in follow[lhs]:
                if symbol in first[rhs[0]]:
                    parse_table[lhs][symbol] = rhs
                else:
                    parse_table[lhs][symbol] = "EPSILON"
        return parse_table


    def calculate(self):
        
        self.eleftrecusrion = self.eliminate_left_recursion(self.productions)
        #self.eleftfctoring = self.eliminate_left_factoring(self.eleftrecusrion)
        self.finalgrammer = self.eleftrecusrion
        
        firstset = self.first(self.finalgrammer)
        followset = self.follow(self.finalgrammer,firstset)
        
        parsetable = self.create_parse_table(self.finalgrammer,firstset,followset)
        
        print("")
        print(f'First Set {firstset}\nFollow Set {followset}')
        print("\nParse Table")
        print(parsetable)

                
        
    
# Example usage
grammar = [
    ("E", "E+T"),
    ("T", "T*F"),
    ("F", "id"),
]
compiler = Compiler(grammar)
compiler.calculate()









First Set {'F': {'i'}, 'E': {'+'}, "E'": {'E'}, 'T': {'*'}, "T'": {'T'}, '-': {'-'}, '+': {'+'}, '*': {'*'}, '(': {'('}, ')': {')'}}
Follow Set {'F': set(), 'i': {'i'}, 'E': set(), '+': {'+'}, "E'": set(), 'T': set(), '*': {'*'}, "T'": set()}

Parse Table
{'F': {}, 'E': {}, "E'": {}, 'T': {}, "T'": {}}
